In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [1]:
!pip install transformers
!pip install google_play_scraper
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.3 MB/s eta 0:00:00


# Import Library

In [7]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments, pipeline, BertModel
from google_play_scraper import app, reviews_all, reviews, Sort
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.mode.chained_assignment = None
np.random.seed(0)
pd.set_option('display.max_columns', None)
import datetime as dt
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud

import csv
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Load Scrapping Genshin Impact dari Play Store

In [ ]:
scrap_ml = reviews_all(
    'com.miHoYo.GenshinImpact',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=1000
)
df_scrap = pd.read_csv()

# Load Scrapping Genshin Impact dari Play Store

In [ ]:
df_scrap = pd.read_csv('hasil scrapping')
df_scrap.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,8041f57b-6f49-4065-9447-a7add558a1dc,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Bug mulu tiap lawan dvalin (sekarang tambah si...,3,0,None,2021-09-03 14:25:58,None,NaT,None
1,5bc6eece-7480-4c81-93cf-039613af1af2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Dari segi grafik bagus banget, hampir semua sp...",5,109,5.0.0_26041933_26161852,2024-08-28 00:05:25,None,NaT,5.0.0_26041933_26161852
2,60bc7037-ad55-4dc6-a271-e0182ad5759d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Game udah bagus, tapi setiap update ada bebera...",3,8,5.0.0_26041933_26161852,2024-08-28 10:21:39,None,NaT,5.0.0_26041933_26161852
3,a61ed2c4-5529-4a0d-8030-c77396e43f03,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Tolong tim hoyoverse untuk menambahkan fitur s...,2,8,5.0.0_26041933_26161852,2024-08-28 03:31:33,None,NaT,5.0.0_26041933_26161852
4,c8dbc71a-a681-46ec-aaba-8c1d6b052446,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Kenapa fitur custom artefak kayak gitu njir, p...",4,4,5.0.0_26041933_26161852,2024-08-29 04:33:59,None,NaT,5.0.0_26041933_26161852


# Pre-processing Data

In [ ]:
clean_df = df_scrap.drop_duplicates()
clean_df = clean_df.dropna(subset=['content'])
n_ulasan, n_kolom = clean_df.shape
print(f'Jumlah ulasan: {n_ulasan}')
print(f'Jumlah kolom: {n_kolom}')

Jumlah ulasan: 158618
Jumlah kolom: 11


# Pre-processing Text

In [ ]:
def cleaning_text(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text)
  text = re.sub(r'#[A-Za-z0-9]+', '', text)
  text = re.sub(r'RT[\s]', '', text)
  text = re.sub(r'http\S+', '', text)
  text = re.sub(r'[0-9]+', '', text)
  text = re.sub(r'[^\w\s]+', '', text)

  text = text.replace('\n', ' ')
  text = text.lower()
  text = text.translate(str.maketrans('', '', string.punctuation))
  return text

def tokenizing(text):
  text = word_tokenize(text)
  return text

def filtering_text(text):
  factory = StopWordRemoverFactory()
  stopword_remover = factory.get_stop_words()
  list_exception = ['tidak']
  stopword_remover.extend(['iya', 'bs','bsa','dr','cmn','dah','yt','gm','gmn','dri' ,'eh','gatau' ,'trs','trus','tuh','utk','wkwk' 'yaa','yg','ny','yah','yng','ajah','min' ,'nya', 'na', 'sih' 'ku','aja', 'di',"ya","loh","kah", 'wkwkwk','jan', 'woi', 'jdi','jd','tp', 'ttp','tpi' ,'wkwk'])

  list_stopwords = stopwords.words('indonesian')
  list_stopwords2 = stopwords.words('english')
  list_stopwords.extend(list_stopwords2)
  stopword_remover.extend(list_stopwords)

  filtered = []
  for word in text:
    if word not in stopword_remover:
      filtered.append(word)
    if word in list_exception:
      filtered.append(word)
  # filtered = ' '.join(text)
  # filtered = stopword_remover.remove(filtered)
  return filtered

def stemming_text(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  words = text.split()
  stemmed = []
  for word in words:
    stemmed.append(stemmer.stem(word))
  stemmed_text = ' '.join(stemmed)
  return stemmed_text

def to_sentences(list_words):
  sentences = ' '. join(word for word in list_words)
  return sentences

slangwords ={"@": "di", "abis": "habis", "wtb": "beli", 'gak':'tidak', "gk": "tidak", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal", "woy": "wahai", 'woi' : 'wahai'}
def fix_slang(text):
  words = text.split()
  fixed_words = []
  for word in words:
    if word in slangwords:
      fixed_words.append(slangwords[word])
    else:
      fixed_words.append(word)
  fixed_text = ' '.join(fixed_words)
  return fixed_text


In [ ]:
df_reviews = clean_df.copy()
df_reviews['text_slangwords'] = df_reviews['content'].apply(fix_slang)
df_reviews['clean_text'] = df_reviews['text_slangwords'].apply(cleaning_text)
df_reviews['tokenized_text'] = df_reviews['clean_text'].apply(tokenizing)
df_reviews['filtered_text'] = df_reviews['tokenized_text'].apply(filtering_text)
df_reviews['final_teks'] = df_reviews['filtered_text'].apply(to_sentences)
df_reviews.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_slangwords,clean_text,tokenized_text,filtered_text,final_teks
0,8041f57b-6f49-4065-9447-a7add558a1dc,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Bug mulu tiap lawan dvalin (sekarang tambah si...,3,0,None,2021-09-03 14:25:58,None,NaT,None,Bug mulu tiap lawan dvalin (sekarang tambah si...,bug mulu tiap lawan dvalin sekarang tambah sig...,"[bug, mulu, tiap, lawan, dvalin, sekarang, tam...","[bug, mulu, lawan, dvalin, signora, gambarnya,...",bug mulu lawan dvalin signora gambarnya pecah ...
1,5bc6eece-7480-4c81-93cf-039613af1af2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Dari segi grafik bagus banget, hampir semua sp...",5,109,5.0.0_26041933_26161852,2024-08-28 00:05:25,None,NaT,5.0.0_26041933_26161852,"Dari segi grafik bagus banget, hampir semua sp...",dari segi grafik bagus banget hampir semua spo...,"[dari, segi, grafik, bagus, banget, hampir, se...","[segi, grafik, bagus, banget, spot, game, kek,...",segi grafik bagus banget spot game kek foto lo...
2,60bc7037-ad55-4dc6-a271-e0182ad5759d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Game udah bagus, tapi setiap update ada bebera...",3,8,5.0.0_26041933_26161852,2024-08-28 10:21:39,None,NaT,5.0.0_26041933_26161852,"Game udah bagus, tapi setiap update ada bebera...",game udah bagus tapi setiap update ada beberap...,"[game, udah, bagus, tapi, setiap, update, ada,...","[game, udah, bagus, update, data, download, da...",game udah bagus update data download data ekst...
3,a61ed2c4-5529-4a0d-8030-c77396e43f03,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Tolong tim hoyoverse untuk menambahkan fitur s...,2,8,5.0.0_26041933_26161852,2024-08-28 03:31:33,None,NaT,5.0.0_26041933_26161852,Tolong tim hoyoverse untuk menambahkan fitur s...,tolong tim hoyoverse untuk menambahkan fitur s...,"[tolong, tim, hoyoverse, untuk, menambahkan, f...","[tim, hoyoverse, fitur, kendaraan, kereta, kud...",tim hoyoverse fitur kendaraan kereta kuda apap...
4,c8dbc71a-a681-46ec-aaba-8c1d6b052446,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Kenapa fitur custom artefak kayak gitu njir, p...",4,4,5.0.0_26041933_26161852,2024-08-29 04:33:59,None,NaT,5.0.0_26041933_26161852,"Kenapa fitur custom artefak kayak gitu njir, p...",kenapa fitur custom artefak kayak gitu njir pa...,"[kenapa, fitur, custom, artefak, kayak, gitu, ...","[fitur, custom, artefak, kayak, gitu, njir, ud...",fitur custom artefak kayak gitu njir udah berh...


# Crop 25k Data

In [ ]:
df_reviews_crop = df_reviews.sample(16000)
df_reviews_crop.to_csv('/content/drive/MyDrive/My Project Colab/Dicoding Project/NLP/data/data_scrap_GI_cleaned_cropin16k.csv')

# Labelling Data with BERT

In [ ]:
# Memuat model dan tokenizer
model_name = "indobenchmark/indobert-base-p1"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Data untuk prediksi
data = df_reviews_crop['final_teks'].tolist()

# Pipeline untuk klasifikasi
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Melakukan prediksi
results = classifier(data)

label = []
score = []

for x in results:
  label.append(x['label'])
  score.append(x['score'])

df_reviews_crop['label'] = label
df_reviews_crop['score'] = score
df_reviews_crop.head()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_slangwords,clean_text,tokenized_text,filtered_text,final_teks,label
64266,b457e58b-6a5c-420b-8600-74256652875f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Karena Game Nya Nge Bosenin, Gacha Nya Ampas M...",0.433722,0,2.2.0_4340077_4464168,2021-11-25 12:34:18,None,NaT,2.2.0_4340077_4464168,"Karena Game Nya Nge Bosenin, Gacha Nya Ampas M...",karena game nya nge bosenin gacha nya ampas mulu,"[karena, game, nya, nge, bosenin, gacha, nya, ...","[game, nge, bosenin, gacha, ampas, mulu]",game nge bosenin gacha ampas mulu,LABEL_0
99794,4166bcac-093a-4896-b61c-182950afb9f5,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,⭐⭐⭐⭐⭐⭐⭐⭐,0.382088,0,2.7.0_6933856_6989239,2022-07-13 00:34:41,None,NaT,2.7.0_6933856_6989239,⭐⭐⭐⭐⭐⭐⭐⭐,,[],[],,LABEL_1
87988,1666d04e-3f2c-47ae-acaf-3545e93f3d21,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,the best.,0.383319,0,4.1.0_18054760_18121248,2023-10-23 07:43:44,None,NaT,4.1.0_18054760_18121248,the best.,the best,"[the, best]",[best],best,LABEL_0
11008,eba441f3-0834-4b90-a970-df66292e2fe0,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Emm gw mantan player genshin dan selebihnya ga...,0.430907,0,None,2021-10-01 01:11:03,None,NaT,None,Emm gw mantan player genshin dan selebihnya ga...,emm gw mantan player genshin dan selebihnya ga...,"[emm, gw, mantan, player, genshin, dan, selebi...","[emm, gw, mantan, player, genshin, selebihnya,...",emm gw mantan player genshin selebihnya game t...,LABEL_0
155337,dd3c3e71-0880-483a-8573-616732e54236,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,So goods,0.374245,0,None,2021-02-10 07:49:32,None,NaT,None,So goods,so goods,"[so, goods]",[goods],goods,LABEL_1


# Ekstraksi Fitur dengan BERT

In [ ]:
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = BertModel.from_pretrained("indobenchmark/indobert-base-p1")
def extract_features(text, model, tokenizer):
    # Tokenisasi teks
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding=True)

    # Dengan torch no_grad untuk menonaktifkan komputasi gradien selama inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Mengambil hidden states dari lapisan terakhir
    last_hidden_states = outputs.last_hidden_state

    # Mengambil embedding dari token [CLS] sebagai representasi teks
    cls_embedding = last_hidden_states[:, 0, :].squeeze().numpy()

    return cls_embedding

features_bert = np.array([extract_features(text, model, tokenizer) for text in df_reviews_crop['final_teks'] ])
print(features_bert)

[[ 1.1705914   1.6510744   0.23546578 ...  0.60678506  0.22255884
   0.16586557]
 [-0.35864136  0.44506493  0.9637301  ...  1.7167472   0.33821407
  -0.11411048]
 [ 0.4376445   1.1870937   1.4037164  ...  1.4957391  -0.3100102
   0.42965588]
 ...
 [ 0.99823177  1.4498922   0.5013755  ...  1.9854748  -0.21103913
   0.65876406]
 [ 0.13902102  1.3388641   1.3220685  ...  1.6383668  -0.45130134
   0.55554485]
 [-0.01244541  1.8618726   0.7465061  ...  2.2124286  -0.10931963
   0.03062695]]


# Klasifikasi dengan Machine Learning

## Skenario Percobaan
1. Ekstraksi Fitur BERT dan Klasifikasi SVM kernel Radial Basis Function
2. Ekstraksi Fitur BERT dan Klasifikasi Logistic Regresion dengan One Vs Rest
3. Ekstraksi Fitur BERT dan Klasifikasi Decision Tree

### Skenario 1 : Ekstraksi Fitur BERT dan Klasifikasi SVM kernel Radial Basis Function

### Membagi Dataset

In [ ]:
# mengambil data dari hasil extraksi fitur bert
x = features_bert
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_reviews_crop['label'])
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

### Training Model

In [ ]:
classifier_rbf = SVC(kernel='rbf')
classifier_rbf.fit(X_train, y_train)
y_pred = classifier_rbf.predict(X_test)

### Performa Model

In [ ]:
print(classification_report(y_test, y_pred, target_names=["Negatif", "Netral", "Positif"]))

              precision    recall  f1-score   support

     Negatif       0.96      0.97      0.97      2152
      Netral       0.95      0.94      0.95       687
     Positif       0.89      0.85      0.87       361

    accuracy                           0.95      3200
   macro avg       0.93      0.92      0.93      3200
weighted avg       0.95      0.95      0.95      3200



### Skenario 2 : Ekstraksi Fitur BERT dan Klasifikasi Logistic Regresion dengan One Vs Rest


### Membagi Dataset

In [ ]:
# mengambil data dari hasil extraksi fitur bert
x = features_bert
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_reviews_crop['label'])
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

### Training Model

In [ ]:
classifier_ovr = LogisticRegression(multi_class='ovr', solver='liblinear')
classifier_ovr.fit(X_train, y_train)
y_pred = classifier_ovr.predict(X_test)

### Performa Model

In [ ]:
print(classification_report(y_test, y_pred, target_names=["Negatif", "Netral", "Positif"]))

              precision    recall  f1-score   support

     Negatif       0.97      0.97      0.97      2152
      Netral       0.95      0.95      0.95       687
     Positif       0.89      0.88      0.89       361

    accuracy                           0.96      3200
   macro avg       0.94      0.94      0.94      3200
weighted avg       0.96      0.96      0.96      3200



### Skenario 3 :Ekstraksi Fitur BERT dan Klasifikasi Decision Tree

### Membagi Dataset

In [ ]:
# mengambil data dari hasil extraksi fitur bert
x = features_bert
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_reviews_crop['label'])
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

### Training Model

In [ ]:
classifier_nb = DecisionTreeClassifier()
classifier_nb.fit(X_train, y_train)
y_pred = classifier_nb.predict(X_test)

### Performa Model

In [ ]:
print(classification_report(y_test, y_pred, target_names=["Negatif", "Netral", "Positif"]))

              precision    recall  f1-score   support

     Negatif       0.91      0.89      0.90      2152
      Netral       0.87      0.86      0.87       687
     Positif       0.60      0.69      0.64       361

    accuracy                           0.86      3200
   macro avg       0.79      0.81      0.80      3200
weighted avg       0.87      0.86      0.86      3200



# Inference Model

In [ ]:
target_names=["Negatif", "Netral", "Positif"]
random_sentiment = "Game Ga guna banget, dikit-dikit bug"

random_sentiment_cleaning = to_sentences(filtering_text(tokenizing(cleaning_text(fix_slang(random_sentiment)))))

y_pred = classifier_rbf.predict(extract_features(random_sentiment_cleaning, model, tokenizer).reshape(1, -1))
print(f"Dari teks {random_sentiment} dikategorikan sebagai = {target_names[y_pred[0]]}")

Dari teks Game Ga guna banget, dikit-dikit bug dikategorikan sebagai = Negatif


# Ekstrak Library

In [ ]:
!pip freeze requirements.txt